In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MODELS = {
    "UI-TARS 7B SFT": "bytedance-research/UI-TARS-7B-SFT"
}

In [ ]:
def load_model(model_path):
    """Load model and tokenizer with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")

    """Load model and tokenizer."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, tokenizer

In [ ]:
def evaluate_qa(model, tokenizer, dataset_name="squad"):
    """Evaluate on SQuAD for QA."""
    dataset = load_dataset(dataset_name, split="validation[:5]")
    metric = load_metric("squad_v2")

    for example in dataset:
        input_text = f"Q: {example['question']}\nContext: {example['context'][:100]}...\nA:"
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print("\nGenerated Answer:", result)
        print("Ground Truth:", example['answers']['text'])
        print("Context:", example['context'][:100], "...\n")

        metric.add(predictions={"id": example["id"], "prediction_text": result, "no_answer_probability": 0.0},
                   references={"id": example["id"], "answers": example["answers"]})

    return metric.compute()

In [ ]:
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, tokenizer = load_model(model_path)

    print("SQuAD QA:", evaluate_qa(model, tokenizer))


Evaluating UI-TARS 7B SFT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]


Generated Answer: Q: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...
A: The Denver Broncos represented the AFC at Super Bowl 50. They played against the Carolina Panthers, who represented the NFC. The Broncos won the game 24-10. Denver's last Super Bowl victory was in Super Bowl 24 in 1990. Denver's last Super Bowl appearance was in Super Bowl 50. Denver's last Super Bowl loss was in Super Bowl 24. Denver's last Super Bowl win was in Super Bowl 24. Denver
Ground Truth: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu ...


Generated Answer: Q: Which NFL team represented the NFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...
A: The San Francisco 49ers represented the NFC at